In [1]:
import pandas as pd

In [2]:
nyc_emb1_path = '../data/nyc/graph/nyc_geohash_node_features.csv'
nyc_emb2_path = '../data/nyc/graph/nyc_geohash_timebase_node.csv'

tky_emb1_path = '../data/tky/graph/tky_geohash_node_features.csv'
tky_emb2_path = '../data/tky/graph/tky_geohash_timebase_node.csv'

nyc_acc = pd.read_csv(nyc_emb1_path)
nyc_time = pd.read_csv(nyc_emb2_path)
tky_acc = pd.read_csv(tky_emb1_path)
tky_time = pd.read_csv(tky_emb2_path)

In [3]:
display(nyc_acc.shape)
display(nyc_time.shape)

display(tky_acc.shape)
display(tky_time.shape)

(1094, 22)

(1101, 49)

(1091, 22)

(1092, 49)

In [4]:
category_cols = [col for col in nyc_time.columns if 'major_category' in col]
# 0 전처리

for col in category_cols:
    nyc_time[col] = nyc_time[col].astype(str)  # 문자열 통일
    nyc_time[col] = nyc_time[col].replace({'0': 'Unknown', '0.0': 'Unknown', 'nan': 'Unknown'})
    print(nyc_time[col].value_counts())
    print('=' * 50)

major_category_weekday
Commercial/Services     353
Residential             255
Transportation          242
Culture & Leisure       113
Healthcare & Welfare     77
Educational              52
Unknown                   9
Name: count, dtype: int64
major_category_holiday
Commercial/Services     349
Residential             252
Transportation          202
Culture & Leisure       133
Unknown                  81
Healthcare & Welfare     61
Educational              23
Name: count, dtype: int64
major_category_morning
Commercial/Services     268
Transportation          245
Unknown                 192
Residential             186
Culture & Leisure        87
Healthcare & Welfare     67
Educational              56
Name: count, dtype: int64
major_category_afternoon
Commercial/Services     346
Transportation          215
Residential             203
Culture & Leisure       127
Unknown                 107
Healthcare & Welfare     60
Educational              43
Name: count, dtype: int64
major_category_eve

In [5]:
# 메인 카테고리 추출
def get_major_category_embedding(df):
    category_cols = ['geohash'] + [col for col in df.columns if 'major_category' in col]
    
    df[category_cols] = df[category_cols].astype(str)
    
    main_cat_df = df[category_cols].copy()
    
    df = df.drop(columns=category_cols[1:])
    
    return df, main_cat_df

nyc_time, nyc_main_cat = get_major_category_embedding(nyc_time)
tky_time, tky_main_cat = get_major_category_embedding(tky_time)

In [6]:
nyc_main_cat 

,geohash,major_category_weekday,major_category_holiday,major_category_morning,major_category_afternoon,major_category_evening,major_category_night
0,dr5nzz,Educational,Educational,Educational,Unknown,Unknown,Unknown
1,dr5ppn,Commercial/Services,Commercial/Services,Commercial/Services,Commercial/Services,Commercial/Services,Unknown
2,dr5ppt,Transportation,Transportation,Transportation,Transportation,Transportation,Transportation
3,dr5pqc,Transportation,Transportation,Unknown,Unknown,Transportation,Transportation
4,dr5pqe,Residential,Residential,Residential,Residential,Residential,Residential
...,...,...,...,...,...,...,...
1096,dr788j,Commercial/Services,Commercial/Services,Unknown,Commercial/Services,Commercial/Services,Unknown
1097,dr788u,Commercial/Services,Commercial/Services,Unknown,Commercial/Services,Commercial/Services,Unknown
1098,dr789p,Culture & Leisure,Culture & Leisure,Transportation,Commercial/Services,Culture & Leisure,Culture & Leisure
1099,dr78c0,Culture & Leisure,Culture & Leisure,Commercial/Services,Commercial/Services,Culture & Leisure,Culture & Leisure


* 임베딩 결합

In [7]:
nyc_emb = pd.merge(nyc_time, nyc_acc, on='geohash', how='outer')
tky_emb = pd.merge(tky_time, tky_acc, on='geohash', how='outer')

nyc_emb.fillna(0, inplace=True)
tky_emb.fillna(0, inplace=True)

display(nyc_emb.head())
display(tky_emb.head())

,geohash,Commercial/Services_weekday,Culture & Leisure_weekday,Educational_weekday,Healthcare & Welfare_weekday,Residential_weekday,Transportation_weekday,count_weekday,Commercial/Services_holiday,Culture & Leisure_holiday,...,highway_trunk,highway_trunk_link,highway_unclassified,oneway_False,oneway_True,bridge_no,bridge_yes,tunnel_building_passage,tunnel_no,tunnel_yes
0,dr5nzz,0.0,0.0,1.0,0.0,0.0,0.0,20.0,0.0,0.0,...,0.0,0.0,0.0,14.0,6.0,18.0,2.0,0.0,20.0,0.0
1,dr5ppn,1.0,0.0,0.0,0.0,0.0,0.0,15.0,1.0,0.0,...,0.0,0.0,0.0,12.0,4.0,14.0,2.0,0.0,14.0,2.0
2,dr5ppt,0.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dr5pqc,0.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0,0.0,...,0.0,0.0,0.0,102.0,2.0,104.0,0.0,0.0,104.0,0.0
4,dr5pqe,0.0,0.0,0.0,0.0,1.0,0.0,15.0,0.0,0.0,...,0.0,0.0,0.0,94.0,13.0,107.0,0.0,0.0,107.0,0.0


,geohash,Commercial/Services_weekday,Culture & Leisure_weekday,Educational_weekday,Healthcare & Welfare_weekday,Residential_weekday,Transportation_weekday,count_weekday,Commercial/Services_holiday,Culture & Leisure_holiday,...,highway_trunk,highway_trunk_link,highway_unclassified,oneway_False,oneway_True,bridge_no,bridge_yes,tunnel_building_passage,tunnel_no,tunnel_yes
0,xn74r5,1.000000,0.0,0.000000,0.000000,0.0,0.000000,5.0,1.000000,0.000000,...,0.0,0.0,164.0,268.0,13.0,275.0,6.0,0.0,281.0,0.0
1,xn74r7,1.000000,0.0,0.000000,0.000000,0.0,0.000000,4.0,0.000000,0.000000,...,0.0,0.0,104.0,224.0,0.0,224.0,0.0,0.0,224.0,0.0
2,xn74r9,0.235294,0.0,0.000000,0.026144,0.0,0.738562,153.0,0.423077,0.000000,...,12.0,0.0,34.0,232.0,17.0,235.0,14.0,0.0,249.0,0.0
3,xn74rc,0.563636,0.0,0.436364,0.000000,0.0,0.000000,55.0,0.666667,0.000000,...,0.0,0.0,68.0,276.0,5.0,273.0,8.0,0.0,281.0,0.0
4,xn74rd,0.000000,0.0,0.000000,0.000000,0.0,1.000000,66.0,0.000000,0.225806,...,0.0,0.0,90.0,300.0,2.0,294.0,8.0,0.0,300.0,2.0


In [8]:
nyc_emb.head()

,geohash,Commercial/Services_weekday,Culture & Leisure_weekday,Educational_weekday,Healthcare & Welfare_weekday,Residential_weekday,Transportation_weekday,count_weekday,Commercial/Services_holiday,Culture & Leisure_holiday,...,highway_trunk,highway_trunk_link,highway_unclassified,oneway_False,oneway_True,bridge_no,bridge_yes,tunnel_building_passage,tunnel_no,tunnel_yes
0,dr5nzz,0.0,0.0,1.0,0.0,0.0,0.0,20.0,0.0,0.0,...,0.0,0.0,0.0,14.0,6.0,18.0,2.0,0.0,20.0,0.0
1,dr5ppn,1.0,0.0,0.0,0.0,0.0,0.0,15.0,1.0,0.0,...,0.0,0.0,0.0,12.0,4.0,14.0,2.0,0.0,14.0,2.0
2,dr5ppt,0.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dr5pqc,0.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0,0.0,...,0.0,0.0,0.0,102.0,2.0,104.0,0.0,0.0,104.0,0.0
4,dr5pqe,0.0,0.0,0.0,0.0,1.0,0.0,15.0,0.0,0.0,...,0.0,0.0,0.0,94.0,13.0,107.0,0.0,0.0,107.0,0.0


In [9]:
nyc_emb.to_csv('../data/nyc/graph/nyc_geohash_basic_embedding.csv', index=False)
tky_emb.to_csv('../data/tky/graph/tky_geohash_basic_embedding.csv', index=False)

nyc_main_cat.to_csv('../data/nyc/graph/nyc_geohash_main_category.csv', index=False)
tky_main_cat.to_csv('../data/tky/graph/tky_geohash_main_category.csv', index=False)